# Exercise 8: Supervised Learning

In this exercise a few basic supervised learning pipelines are elaborated.
* Data analysis
* Feature engineering
* Cross-validation
* Regression
  * Linear models and neural networks
* Classification
  * Linear models and gradient boosting machines

Moreover, working with the popular __pandas__ library is presented here.

## Load Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
import seaborn as sns
plt.style.use('dark_background')

## Data Set
In this exercise we will use a [data set hosted on Kaggle](https://www.kaggle.com/hankelea/system-identification-of-an-electric-motor).
For ease of use and simplicity, we are going to work on a reduced version though, that is provided with this notebook already.

The data set represents lab measurements from a test bench, where a three-phase electric motor was externally speed-controlled while being induced with different voltages, leading to corresponding currents and, thus, torque on the motor.

### Context
The most important aspect of electric vehicles is their efficiency or achievable range. 
In order to achieve high efficiency, it is essential to avoid the over-dimensioning of the drive train during the design phase.
Therefore, the drive train has to be kept as lightweight as possible while at the same time it should be utilized to its maximum extent. 
This can only be achieved if the dynamic behavior of the drive train is accurately known by the controller/agent. 
The task of the controller is to achieve a desired torque at the wheels of the car by controlling the currents of the electric motor.

Control, in this context, is about choosing appropriate, discrete switching states for the transistors in the attached power electronic unit, that affect the voltage applied to the motor.
Knowing which switching state leads to what current change at each point in time helps the controller to choose an appropriate action.
Since in the real world many parasitic, intangible and volatile effects are imposed on currents formation, modeling them statistically is an auspicious endeavor.

### Features

Most of the time, a data set is a table/matrix/2d-tensor with labeled columns (and rows).

Here, rows are consecutively enumerated while columns are the following:

* __n_1k__: elementary vector applied between (k-1) and k. The elementary vector denotes the label-encoded switching state in the converter.
* __id_k__: d-current at k in A (Ampere).
* __iq_k__: q-current at k in A (Ampere). 
* __epsilon_k__: rotor angle at k in rad (Radian).
* __n_k__: elementary vector applied between k and (k+1). 
* __id_k1__: d-current at (k+1) in A (Ampere).
* __iq_k1__: q-current at (k+1) in A (Ampere).

[Read up on d/q-transformation](https://en.wikipedia.org/wiki/Direct-quadrature-zero_transformation) to understand the coordinate notation of the currents (assuming the amplitude-invariant transformation than the power-invariant).

The enlisted features will be utilized for either regression or classification.
Depending on the task, the features denoting input and target quantities will vary, such that the specific mapping will be discussed later.

Lab measurements from a motor are commonly time series data.
Yet, here the records are shuffled, and we only have the link between two elementary vectors (switching states/ actions) in a row itself.

### Optional Literature
If you find the following exercise intriguing and you would like to keep on investigating, have a look at the scientific papers that describe the background to the underlying data set:
* [Part 1](https://arxiv.org/pdf/2003.07273.pdf)
* [Part 2](https://arxiv.org/pdf/2003.06268.pdf)

### Load CSV data set

In [ ]:
df = pd.read_csv('reduced_emotor.csv')
print(df.shape)
df.head()

The pandas library makes reading csv files easy, next to its superb data wrangling utilities. 

Think of pandas as a layer on top of numpy, that is optimized for tabular data and attaches column/row names to a matrix.

The main pandas class is the so-called _DataFrame_ which holds a table, and the _Series_  that denotes a single column.
Both classes do not only consist of arrays representing the data itself but also of an index (row labeling), that affects basic arithmetic operations between different instantiated objects.

Please refer to the pandas notebooks from exercise 1 for a refresher.

### Adding new features
There are two main ways to add further features to the dataframe.

In [ ]:
# new feature "pairs"
df = df.assign(pairs=lambda r: r.n_1k.astype(str)+'->'+r.n_k.astype(str))
# alternatively
df['pairs_alternative'] = df.n_1k.astype(str) + '->' + df.n_k.astype(str) 

assert all(df.pairs == df.pairs_alternative)
# remove a feature
del df['pairs_alternative']
# alternatively
# df = df.drop(['pairs_alternative'], axis=1)

In [ ]:
df.columns

In [ ]:
df.pairs.value_counts().sort_index().head(10)

We recognize, that all transitions within a row appear uniformly throughout the dataset, which is not the case for the original kaggle data but has been intentionally rectified for this exercise's reduced version.

## Exploratory Data Analysis
Before any feature engineering should be conducted, it is advisable to investigate the general structure of the data.

This process is called exploratory data analysis (EDA) and usually comprises 
* distribution visualizations, 
* missing data detection, 
* substitution of such, 
* linear correlation analysis,
* time series analysis
* etc.

### Distribution visualizations

In [ ]:
def plot_dist(_df):
    fig, axes = plt.subplots(1,2,sharex=True, sharey=True)
    for c, ax in zip(['n_k', 'n_1k'], axes.flatten()):
        sns.countplot(x=c, data=_df,  ax=ax)

In [ ]:
plot_dist(df)

Each elementary vector occurs around 350 thousand times for both, at time _k_ and time _k-1_ .

In the following we plot the distribution of the currents and the motor angle unfolded over each elementary vector.

For this we utilize pandas' _groupby()_ function, which helps to group (and even aggregate) the complete data set with respect to a categorical feature.

In [ ]:
analyzed_cols = [c for c in df if c not in ['n_k', 'n_1k', 'pairs']]

# prepare colors
color_list = plt.cm.Set3(np.linspace(0, 1, 12))
color_d = {'id_k': rgb2hex(color_list[0]),
           'id_k1': rgb2hex(color_list[0]),
           'iq_k': rgb2hex(color_list[1]),
           'iq_k1': rgb2hex(color_list[1]),
           'epsilon_k': rgb2hex(color_list[2])}

def dist_plot(dframe, cols_to_plot):
    unique_elem_vecs = dframe['n_k'].nunique()
    # create plot
    fig, axes = plt.subplots(nrows=unique_elem_vecs, ncols=len(cols_to_plot), 
                             sharex='col', sharey='col', figsize=(3*unique_elem_vecs, 10))
    for k, _df in dframe.groupby('n_k'):
        for i, c in enumerate(cols_to_plot):
            sns.distplot(_df[c], ax=axes[k-1, i], color=color_d.get(c, None))
            if i == 0:
                axes[k-1, i].set_ylabel(f'n_k = {k}')
            if k == 1:
                axes[k-1, i].set_title(c)
            xlbl = c if k == 7 else ''
            axes[k-1, i].set_xlabel(xlbl)

    plt.tight_layout()
dist_plot(df, analyzed_cols)

_(Double-click picture if it appears too small)_.

It seems that _epsilon_k_ exhibits a sine shaped histogram while its range covers $[-\pi, \pi]$.

More subtle, we recognize a semi-sphere shape of the 2d histogram between the currents (remember, d and q currents are to be plotted perpendicular to each other).
It might be auspicious, to add another feature denoting the current vector norm sqrt(id^2 + iq^2).
From the principle of d/q-transformation, we know that the $\ell_2$-norm of the d- and q-current denotes the current amplitude in the system.
From that we recognize, that smaller current magnitudes seem more likely than higher.

#### Task 1: Add features and plot their distributions
* Add sine and cosine of the rotor angle to the dataframe
* Add the current vector norm of both time steps to the dataframe: $i_{norm} = \sqrt{i_d^2 + i_q^2}$
* Plot their distributions

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Scatter plots
Probably, the previous currents are good indicators for the next currents.
Let's visualize this relationship unfold over each transition.

In [ ]:
n_cols = 7
n_rows = 7
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, 
                         sharex=True, sharey=True, figsize=(20, 20))
df = df.sort_values(by='pairs')
for ax, (k, _df) in zip(axes.flatten(), df.groupby('pairs')):
    t_prev, t = k.split('->')
    ax.scatter(_df.id_k, _df.id_k1, s=.5, label='d')
    ax.scatter(_df.iq_k, _df.iq_k1, s=.5, label='q')
    if t == '1':
        ax.set_ylabel(f'n_1k = {t_prev}\n i_k1 in A')
    if t_prev == '7':
        ax.set_xlabel(f'i_k in A\n n_k = {t}')
    ax.grid(alpha=.5)
plt.tight_layout()

ax.legend(ncol=2, loc='lower center', bbox_to_anchor=(.5, 1), bbox_transform=fig.transFigure, markerscale=6)

Apparently, transitions to switching_state == 1 come with far less scatter than for other switching states.
Beyond this, the overall scatter seems relatively linear.

We investigate this by a linear correlation heat map.

### Linear Correlation
The [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) indicates the strength of the linear correlation between two random variables.

In [ ]:
corr = df.corr()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize=(14,14))
_ = sns.heatmap(corr, mask=mask, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .8})

It becomes obvious, that features from previous time steps exhibit the strongest linear correlation with those of the actual time step.

This suggests a strong timely dependency, currents follow a [non-stationary process](https://en.wikipedia.org/wiki/Stationary_process).

It is very likely, that more history information for each point in time would reveal precious patterns for an exact forecast of the currents, but these are not available here.

__Bonus task__ : Can you reconstruct measurement sessions from the shuffled dataset by comparison of samples? Use the original data set for this.

## Cross Validation

Before we start modeling, we need to agree on a certain cross-validation (CV) technique.
We will choose a method from scikit-learn.

Please install it with
```
pip install scikit-learn
```

Since distribution of elementary vectors is rather balanced, we will go with a simple CV: 5-Fold CV.

Recall from the lecture what K-Fold CV stands for.

![](kfold-cv.png)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5, shuffle=True)

## Regression
We start off with a regression example: Given the data set, predict the future currents at any time.

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse


# more feature engineering
enriched_df = df.assign(id_k_sqrd=lambda x: x.id_k**2,
                        iq_k_sqrd=lambda x: x.iq_k**2,
                        iq_x_id_k=lambda x: x.id_k*x.iq_k,
                        i_norm_sqrd=lambda x: x.i_norm_k**2,
                        id_k_log=lambda x: np.log1p(x.id_k.abs()),
                        iq_k_log=lambda x: np.log1p(x.iq_k),
                        id_k_exp=lambda x: np.exp(-x.id_k.abs()),
                        iq_k_exp=lambda x: np.exp(-x.iq_k.abs()),
                        i_norm_log=lambda x: np.log1p(x.i_norm_k),
                        i_norm_exp=lambda x: np.exp(x.i_norm_k.abs()),
                        id_plus_iq=lambda x: x.id_k+x.iq_k,
                        id_minus_iq=lambda x: x.id_k-x.iq_k,
                        id_over_iq=lambda x: x.id_k/ (x.iq_k + 1),
                        iq_over_id=lambda x: x.iq_k/ (x.id_k - 1),
                        # the following features encompass the whole data set. Is this problematic?
                        id_pairgroup_normed=lambda x: x.id_k - df.groupby('pairs')['id_k'].transform('mean'),
                        iq_pairgroup_normed=lambda x: x.iq_k - df.groupby('pairs')['iq_k'].transform('mean'),
                        id_ngroup_normed=lambda x: x.id_k - df.groupby('n_k')['id_k'].transform('mean'),
                        iq_ngroup_normed=lambda x: x.iq_k - df.groupby('n_k')['iq_k'].transform('mean'),
                      )

y_cols = ['id_k1', 'iq_k1', 'i_norm_k1']
x_cols = [c for c in enriched_df if c not in y_cols + ['pairs'] ]

for i, (train_index, test_index) in enumerate(kf.split(enriched_df)):
    x_train, y_train = enriched_df.loc[train_index, x_cols], enriched_df.loc[train_index, y_cols]
    x_test, y_test = enriched_df.loc[test_index, x_cols], enriched_df.loc[test_index, y_cols].values
    
    # feature aggregations over the training set
    model = LinearRegression()

    model.fit(x_train, y_train)
    prediction = model.predict(x_test)

    result_string = ', '.join(f'{y_cols[j]}: {mse(y_test[:, j], prediction[:, j]):.2f} A²' for j in range(len(y_cols)))
    print(f'Fold {i}: \nMSE:\t', result_string)

The error is homogenous across folds, which affirms our CV strategy.

However, the estimation error is fairly large.

#### Task 2: What did we miss to do before fitting the linear model?
Two important preprocessing steps were missed before conducting model training.
Which might they be? It has something to do with categorical values and value ranges.

In the following, rectify this blunder, and prepare the data appropriately.
* Conduct the CV again.
* Plot the estimation error of the last fold over the corresponding ground truth.

Can you think of more features? Do you think there might be a problem with those features that were aggregated over the full data set in the beginning?

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

## Neural Networks
In the following, a neural network class is given, which can be used the very same way as the scikit-learn linear model.

For the imports to work, please install tensorflow 2:

```
pip install tensorflow
```


In [ ]:
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as opts
from tensorflow.keras import layers, regularizers, initializers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def build_mlp_model(x_shape=(100, 1, 10),  # input shape to the NN
                  n_layers=1,  # number of hidden layers
                  n_units=64,  # number of neurons per hidden layer
                  kernel_reg=1e-9,  # kernel regularization strength
                  activity_reg=1e-9,  # activity regularization strength
                  bias_reg=1e-9,  # bias regularization strength
                  activation='relu',  # activation function
                  lr_rate=1e-5,  # learning rate (step size)
                  loss='mse',  # cost function for training (here, MSE)
                  n_targets=1,  # number of neurons in output layer
                  seed=None, print_summary=True):
    """build mlp model"""

    cfg = {
        'units': int(n_units),
        'kernel_regularizer': regularizers.l2(kernel_reg),
        'activity_regularizer': regularizers.l2(activity_reg),
        'bias_regularizer': regularizers.l2(bias_reg),
        'kernel_initializer': initializers.lecun_normal(seed=seed),
        'activation': activation,
    }
    
    # keras functional API
    x = layers.Input(shape=x_shape)  # Input layer
    y = layers.Dense(**cfg)(x)  # Hidden layer (feed-forward)
    if n_layers > 1:
        for i in range(n_layers-1):
            y = layers.Dense(**cfg)(y)   # Hidden layer (feed-forward)
    y = layers.Dense(n_targets)(y)  # Output layer (feed-forward)

    model = Model(inputs=x, outputs=y)
    model.compile(optimizer=opts.Adam(lr=lr_rate), loss=loss)
    if print_summary:
        model.summary()
    return model

class MLP(KerasRegressor):
    """Multi-Layer Perceptron with Scikit-learn API"""
    
    def __init__(self, len_x_cols, len_y_cols, print_summary=True):
        # Hard coded build function and kwargs
        KerasRegressor_config = {'batch_size': 128,  # how many samples per batch
                               'epochs': 99,  # amount of iterations over the data set
                               'activation': 'relu',  # activation function
                               'n_layers': 1,  # no. hidden layers
                               'n_units': 32,  # no. hidden units/neurons
                               'kernel_reg': 1e-9,  # kernel l2 regularization strength
                               'activity_reg': 1e-9,  # activity l2 regularization strength
                               'bias_reg': 1e-9,  # bias l2 regularization strength
                               'lr_rate': 5e-4,  # learn rate
                               'n_targets': len_y_cols,  # no. targets
                               'loss': 'mse',  # loss function
                               'verbose': 1,  # verbosity
                               'x_shape': (len_x_cols,),  # input shape into NN
                               'print_summary': print_summary
                               }
        super().__init__(build_fn=build_mlp_model, **KerasRegressor_config)

    def fit(self, X, y, **kwargs):
        
        # Neural Network training encompasses validation against another validation 
        #  set (next to those of kfold CV), on which overfitting is monitored per epoch
        if kwargs.get('validation_data', None) is None:
            # determine validation set dynamically
            val_ratio = 0.1  # 10% validation data
            if isinstance(X, pd.DataFrame): X = X.values
            if isinstance(y, pd.DataFrame): y = y.values
            n_val_set = int(val_ratio * len(X))
            X, X_val = X[:-n_val_set, :], X[-n_val_set:, :]
            if len(y.shape) == 1:
                y = y[:, np.newaxis]
            y, y_val = y[:-n_val_set, :], y[-n_val_set:, :]
            kwargs['validation_data'] = (X_val, y_val)
        
        # Add learn scheduling
        #  Early stopping: Stop training if validation error plateaus
        #  ReduceLrOnPlateau: If loss on train set plateaus, reduce learn rate
        kwargs['callbacks'] = [EarlyStopping(monitor='val_loss',
                                              min_delta=1e-3,
                                              patience=10,
                                              verbose=1),
                                ReduceLROnPlateau(monitor='loss',
                                                  patience=5,
                                                  factor=0.5), ]
        kwargs['epochs'] = 20
        kwargs['batch_size'] = 64
        ret = super().fit(X, y, **kwargs)

        return ret

#### Task 3: Conduct CV under the same regression task as above.
Is a neural network better?

Be aware that training time of a neural network is substantially longer than for linear models. If you face out-of-memory errors, please reduce the n_units parameter, the batch_size parameter, or subsample the data set.

Bonus task: Try to tweak the hyperparameters of the NN in order to achieve a better performance.

If you find a decent setting, you are invited to post your solution to [kaggle kernels](https://www.kaggle.com/hankelea/system-identification-of-an-electric-motor/kernels). Don't be shy!

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Classification
We swap the task from regression to classification by predicting __n_k__ given all other features including currents at time k+1.

We start off with [logistic regression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) (which is a classifying model despite its name).

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, log_loss  # cross-entropy error
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)

# instantiate a logistic regression model with the following arguments for faster training
LogisticRegression(solver='sag', multi_class='multinomial', max_iter=30, n_jobs=-1)

#### Task 4: Conduct CV with LogisticRegression under a classification task
* Predict __n_k__ - all other features are allowed as input features.
Use the [cross-entropy loss](https://scikit-learn.org/stable/modules/model_evaluation.html#log-loss) as metric.
* Print the corresponding [confusion matrix](https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) of the last fold's prediction

Tip: We need to restore the original n_k feature but keep n_1k one-hot-encoded in order to use it as input feature.

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Gradient Boosting Machines

For the classification with GBMs we will utilize [Microsoft's lightGBM](https://lightgbm.readthedocs.io/en/latest/index.html), which comes with a scikit-learn estimator interface:

```
pip install lightgbm
```

In [ ]:
from lightgbm import LGBMClassifier


# initialize like the following
LGBMClassifier(objective='multiclass', n_jobs=-1, n_estimators=30)

#### Task 5: Use a GBM to classify n_k

Recall, that for a decision-tree-based model, normalization is optional.

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Note in particular, runtime and precision differences between GBMs and logistic regression, as well as in resource utilization.